# Open Shell Code tests


Broadly, there are three things we need to do make sure of before we can include the open shell code into the main repo.
1. Closed shell code is not affected in any way by the implementation of open shell.
2. Open shell code replicates closed shell results accurately.
3. Open shell results are accurate.

## 1.

We can check this using the existing tests for closed shell code.

## 2.

Overall
- Run closed shell experiemnts with both open and closed shell methods.
- Hamiltonians are numerically close

Localizers
- Active MOs should be consistent across un/restricted for closed shells.
- 

In [68]:
from nbed.driver import NbedDriver
from pathlib import Path

water_filepath = Path("molecules/water.xyz").absolute()

args = {
    "geometry": str(water_filepath),
    "n_active_atoms": 1,
    "basis": "STO-3G",
    "xc_functional": "b3lyp",
    "projector": "mu",
    "localization": "spade",
    "convergence": 1e-6,
    "savefile": None,
    "run_ccsd_emb": False,
    "run_fci_emb": False,
}

restric_driver = NbedDriver(
    geometry=args["geometry"],
    n_active_atoms=args["n_active_atoms"],
    basis=args["basis"],
    xc_functional=args["xc_functional"],
    projector=args["projector"],
    localization=args["localization"],
    convergence=args["convergence"],
    savefile=args["savefile"],
    run_ccsd_emb=args["run_ccsd_emb"],
    run_fci_emb=args["run_fci_emb"],
    force_unrestricted=False,
)

unrestric_driver = NbedDriver(
    geometry=args["geometry"],
    n_active_atoms=args["n_active_atoms"],
    basis=args["basis"],
    xc_functional=args["xc_functional"],
    projector=args["projector"],
    localization=args["localization"],
    convergence=args["convergence"],
    savefile=args["savefile"],
    run_ccsd_emb=args["run_ccsd_emb"],
    run_fci_emb=args["run_fci_emb"],
    force_unrestricted=True,
)



In [69]:
from numpy import all, allclose
for k in restric_driver.localized_system.__dict__.keys():
    rk = getattr(restric_driver.localized_system, k, 0)
    uk = getattr(unrestric_driver.localized_system, k, 1)
    if not all(rk == uk):
        print(k)

_global_ks
_restricted_scf
enviro_selection_condition
c_active
c_enviro
_c_loc_occ
dm_active
dm_enviro
beta_active_MO_inds
beta_enviro_MO_inds
beta_c_active
beta_c_enviro
_beta_c_loc_occ
beta_dm_active
beta_dm_enviro


In [86]:
rls = restric_driver.localized_system
uls = unrestric_driver.localized_system

# print(allclose(uls.dm_active+uls.beta_dm_active, rls.dm_active))
# print(uls.dm_active+uls.beta_dm_active - rls.dm_active)

# Why would spin up and down coeffs be different?
allclose(uls._global_ks.mo_coeff[0], uls._global_ks.mo_coeff[1], 1e-4)



False


## 3.

This is the trickiest bit
- We can first check that the general behaviour is correct, what do we expect for the +1 and -1 ions of a molecule?
- Check results against physical limits, for instance if any are below the ground state these need to be explained.
- Find published energies of some example ions and compare to these.